In [17]:
import numpy as np
import pandas as pd
from qiskit_optimization import QuadraticProgram

In [18]:
expected_returns = pd.read_csv("results/expected_returns.csv", index_col=0)
cov_matrix = pd.read_csv("results/cov_matrix.csv", index_col=0)

In [19]:

mu = expected_returns.values.flatten()   # expected returns vector
Sigma = cov_matrix.values                # covariance matrix
n_assets = len(mu)

In [29]:
N = 10  # or any small number
mu = mu[:N]
Sigma = Sigma[:N, :N]
n_assets = N

In [40]:
gamma = 0.5

# Flexible budget: you can choose either by number or percentage
# Example: set budget_type = "percentage" and budget_value = 0.3 (30%)
#          or set budget_type = "number" and budget_value = 10 (10 stocks)

budget_type = "number"   # options: "percentage" or "number"
budget_value = 5         # if percentage → fraction of total assets, if number → count

if budget_type == "percentage":
    budget = int(n_assets * budget_value)
else:  # "number"
    budget = int(budget_value)

# Ensure budget is within valid range
budget = max(1, min(budget, n_assets))
print(f"Budget constraint: selecting {budget} out of {n_assets} assets")

Budget constraint: selecting 5 out of 10 assets


In [41]:
qp = QuadraticProgram("Portfolio Optimization")

In [42]:
# Decision variables: x_i = 1 if asset i is chosen, 0 otherwise
for i in range(n_assets):
    qp.binary_var(name=f"x_{i}")

In [43]:
# Objective: maximize returns - gamma * risk
# Equivalent to: minimize -mu^T x + gamma * x^T Σ x
linear = -mu
quadratic = gamma * Sigma
qp.minimize(linear=linear, quadratic=quadratic)

In [44]:
# Budget constraint: select exactly "budget" assets
qp.linear_constraint(
    linear={f"x_{i}": 1 for i in range(n_assets)},
    sense="==",
    rhs=budget,
    name="budget_constraint"
)

<LinearConstraint: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_... == 5 'budget_constraint'>

In [45]:
import numpy as np
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [46]:
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=1)
optimizer = MinimumEigenOptimizer(qaoa)

In [47]:
result = optimizer.solve(qp)

In [48]:
print("\nOptimal portfolio allocation:")
print(result)


Optimal portfolio allocation:
fval=-0.0046530926689078485, x_0=0.0, x_1=0.0, x_2=1.0, x_3=1.0, x_4=0.0, x_5=1.0, x_6=1.0, x_7=0.0, x_8=0.0, x_9=1.0, status=SUCCESS
